In [38]:
import os
import re
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
from metaphone import doublemetaphone
from pathlib import Path
import plotly.io as pio
import plotly.express as px

#for Splink
from splink.exploratory import completeness_chart
from splink.blocking_analysis import count_comparisons_from_blocking_rule, n_largest_blocks,cumulative_comparisons_to_be_scored_from_blocking_rules_chart
from splink.exploratory import profile_columns
import splink.comparison_library as cl
from splink import block_on,Linker, SettingsCreator ,DuckDBAPI


In [2]:
def load_data(file_path:str) -> pd.DataFrame:
    """Load data from csv file
    
    Args:
    file_path: Path to CSV file
    
    Returns:
    pd.DataFrame: Dataframe loaded from csv file"""

    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"{file_path} not found.")
        
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        raise


In [3]:
df_a= load_data('abhidataset.csv')
df_b = pd.DataFrame(columns=df_a.columns)

In [4]:
def preview_data(df: pd.DataFrame):
    """Print basic information and preview of the dataframe."""
    display(df.head())
    print(df.info()) 

preview_data(df_a)
preview_data(df_b)

,full_name,first_and_last_name,first_name,last_name,dob,birth_place,postcode_fake,gender,occupation,email,phone,address,city,country,zip
0,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician,thomaschudleigh2499@gmail.com,6600795731,devon Streeet,DEVON,United Kingdom,tq13 8df
1,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician,thomaschudleigh5161_at_outlook.in,-4615,916 devon,DEVON,United Kingdom,TQ13 8DF
2,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician,tomchudleigh9081@outlook.in,1372822399,022 devon,DEVON,U.K,TQ13 8DF
3,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,\N,politician,thomaschudleigh3478_at_gmail.com,+1-536-360-0037x1027,261 devon,devon,UK,tq138hu
4,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,\N,politician,thomaschudleigh6012@outlook.in,4.49099E+11,17 devon,devon,U.K,TQ13 8DF


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50578 entries, 0 to 50577
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   full_name            50578 non-null  object
 1   first_and_last_name  50578 non-null  object
 2   first_name           50578 non-null  object
 3   last_name            50578 non-null  object
 4   dob                  50578 non-null  object
 5   birth_place          50578 non-null  object
 6   postcode_fake        50578 non-null  object
 7   gender               50578 non-null  object
 8   occupation           50578 non-null  object
 9   email                50578 non-null  object
 10  phone                50578 non-null  object
 11  address              50578 non-null  object
 12  city                 50578 non-null  object
 13  country              50578 non-null  object
 14  zip                  50578 non-null  object
dtypes: object(15)
memory usage: 5.8+ MB
None


,full_name,first_and_last_name,first_name,last_name,dob,birth_place,postcode_fake,gender,occupation,email,phone,address,city,country,zip


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   full_name            0 non-null      object
 1   first_and_last_name  0 non-null      object
 2   first_name           0 non-null      object
 3   last_name            0 non-null      object
 4   dob                  0 non-null      object
 5   birth_place          0 non-null      object
 6   postcode_fake        0 non-null      object
 7   gender               0 non-null      object
 8   occupation           0 non-null      object
 9   email                0 non-null      object
 10  phone                0 non-null      object
 11  address              0 non-null      object
 12  city                 0 non-null      object
 13  country              0 non-null      object
 14  zip                  0 non-null      object
dtypes: object(15)
memory usage: 124.0+ bytes
None


In [5]:
def clean_phone_number(phone):
    """
    Clean and standardize phone numbers to a consistent format.
    Returns a tuple of (cleaned_number, extension)
    
    Args:
        phone: The phone number string to clean
        
    Returns:
        pd.Series: Series with [cleaned_number, extension]
    """
    if pd.isnull(phone):
        return pd.Series([np.nan, np.nan])

    # Convert to string and remove whitespace
    phone_str = str(phone).lower().strip()

    # Regex to split extension (e.g., x123, ext.456)
    match = re.match(r'([^\dxext]*\d[\d\D]*?)(?:\s*(?:ext\.?|x)\s*(\d+))?$', phone_str)

    if match:
        main_part = re.sub(r'\D', '', match.group(1))  # remove non-digits
        extension = match.group(2) if match.group(2) else np.nan
        return pd.Series([main_part, extension])
    else:
        return pd.Series([re.sub(r'\D', '', phone_str), np.nan])


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize and clean input data
    
    Args:
        df: Input DataFrame
        
    Returns:
        pd.DataFrame: Preprocessed DataFrame
    """
    df = df.copy()

    # --- Map user's schema to canonical columns expected downstream ---
    # surname -> last_name
    if 'last_name' not in df.columns and 'surname' in df.columns:
        df['last_name'] = df['surname']
    # address -> street
    if 'street' not in df.columns and 'address' in df.columns:
        df['street'] = df['address']
    # postal_code or postcode_fake -> zip
    if 'zip' not in df.columns:
        if 'postal_code' in df.columns:
            df['zip'] = df['postal_code']
        elif 'postcode_fake' in df.columns:
            df['zip'] = df['postcode_fake']
    # Ensure state column exists for downstream logic
    if 'state' not in df.columns:
        df['state'] = np.nan
    # Ensure first_name exists (user has first_name)
    # Ensure city exists (user has city)
    # Ensure email, phone exist (user has email, phone)

    # Clean text columns
    text_cols = ['first_name', 'last_name', 'street', 'city', 'state', 'email']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower()

    # Format ZIP/Postcodes: normalize case and spacing (supports alphanumeric postcodes)
    if 'zip' in df.columns:
        df['zip'] = df['zip'].astype(str).str.strip()
        df['zip'] = df['zip'].str.replace(r'\s+', '', regex=True).str.upper()

    # Clean phone numbers
    if 'phone' in df.columns:
        df[['phone', 'phone_ext']] = df['phone'].apply(clean_phone_number)

    # Extract email domain
    if 'email' in df.columns:
        df['email'] = df['email'].str.lower()
        df['email_domain'] = df['email'].str.extract(r'@([\w\.-]+)', expand=False)

    # To make entire DataFrame consistent
    df = df.where(pd.notnull(df), np.nan)

    # Create full name if missing
    if 'full_name' not in df.columns and 'first_name' in df.columns and 'last_name' in df.columns:
        df['full_name'] = df['first_name'].fillna('') + ' ' + df['last_name'].fillna('')
    if 'full_name' in df.columns:
        df['full_name'] = df['full_name'].astype(str).str.strip().str.lower()

    # Create phonetic encodings
    if 'first_name' in df.columns:
        df['first_name_metaphone'] = df['first_name'].apply(
            lambda x: doublemetaphone(x)[0] if pd.notnull(x) else np.nan
        )
    
    if 'last_name' in df.columns:
        df['last_name_metaphone'] = df['last_name'].apply(
            lambda x: doublemetaphone(x)[0] if pd.notnull(x) else np.nan
        )

    return df

In [6]:
preprocessed_data_a = preprocess_data(df_a)
# Avoid preprocessing an empty df_b; create an empty frame with expected columns
preprocessed_data_b = pd.DataFrame(columns=preprocessed_data_a.columns)

In [7]:
preview_data(preprocessed_data_a)
preview_data(preprocessed_data_b)

,full_name,first_and_last_name,first_name,last_name,dob,birth_place,postcode_fake,gender,occupation,email,...,address,city,country,zip,street,state,phone_ext,email_domain,first_name_metaphone,last_name_metaphone
0,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician,thomaschudleigh2499@gmail.com,...,devon Streeet,devon,United Kingdom,TQ138DF,devon streeet,nan,NaN,gmail.com,TMS,XTL
1,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician,thomaschudleigh5161_at_outlook.in,...,916 devon,devon,United Kingdom,TQ138DF,916 devon,nan,NaN,NaN,TMS,XTL
2,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician,tomchudleigh9081@outlook.in,...,022 devon,devon,U.K,TQ138DF,022 devon,nan,NaN,outlook.in,TM,XTL
3,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,\N,politician,thomaschudleigh3478_at_gmail.com,...,261 devon,devon,UK,TQ138HU,261 devon,nan,1027,NaN,TMS,XTL
4,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,\N,politician,thomaschudleigh6012@outlook.in,...,17 devon,devon,U.K,TQ138DF,17 devon,nan,NaN,outlook.in,TMS,XTL


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50578 entries, 0 to 50577
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   full_name             50578 non-null  object
 1   first_and_last_name   50578 non-null  object
 2   first_name            50578 non-null  object
 3   last_name             50578 non-null  object
 4   dob                   50578 non-null  object
 5   birth_place           50578 non-null  object
 6   postcode_fake         50578 non-null  object
 7   gender                50578 non-null  object
 8   occupation            50578 non-null  object
 9   email                 50578 non-null  object
 10  phone                 50578 non-null  object
 11  address               50578 non-null  object
 12  city                  50578 non-null  object
 13  country               50578 non-null  object
 14  zip                   50578 non-null  object
 15  street                50578 non-null

,full_name,first_and_last_name,first_name,last_name,dob,birth_place,postcode_fake,gender,occupation,email,...,address,city,country,zip,street,state,phone_ext,email_domain,first_name_metaphone,last_name_metaphone


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   full_name             0 non-null      object
 1   first_and_last_name   0 non-null      object
 2   first_name            0 non-null      object
 3   last_name             0 non-null      object
 4   dob                   0 non-null      object
 5   birth_place           0 non-null      object
 6   postcode_fake         0 non-null      object
 7   gender                0 non-null      object
 8   occupation            0 non-null      object
 9   email                 0 non-null      object
 10  phone                 0 non-null      object
 11  address               0 non-null      object
 12  city                  0 non-null      object
 13  country               0 non-null      object
 14  zip                   0 non-null      object
 15  street                0 non-null      object
 16  st

In [8]:
def combine_dataframes(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """Combine two preprocessed DataFrames, reset index, and add unique ID."""
    combined_df = pd.concat([df1, df2], ignore_index=True)
    combined_df.reset_index(drop=True)
    combined_df["unique_id"] = combined_df.index.astype(str)
    return combined_df

def identify_and_show_duplicates(df: pd.DataFrame, subset_cols: list) -> pd.DataFrame:
    """
    Create a 'duplicate_of' column showing the duplicate occurrences.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        subset_cols (list): List of column names to check for duplicates.
        
    Returns:
        pd.DataFrame: Updated DataFrame where 'duplicate_of' is 2 or more.
    """
    duplicates = df[df.duplicated(subset=subset_cols)]
    
    return duplicates


def analyze_completeness(df: pd.DataFrame, db_api: DuckDBAPI):
    """Generate completeness chart."""
    return completeness_chart(df, db_api=db_api)

def profile_data(df: pd.DataFrame, db_api: DuckDBAPI, top_n: int = 10, bottom_n: int = 5):
    """Profile selected columns using DuckDBAPI."""
    return profile_columns(df, db_api=db_api, top_n=top_n, bottom_n=bottom_n)


In [ ]:
# --- Upload local data to MinIO as Parquet (for Trino/Hive to read) ---
# Requires: pip install minio pyarrow

from minio import Minio
from minio.error import S3Error
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO

# Configure MinIO
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT", "localhost:9000")  # host:port
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.getenv("MINIO_SECRET_KEY", "minioadmin")
MINIO_SECURE = os.getenv("MINIO_SECURE", "false").lower() == "true"  # True if https
BUCKET = os.getenv("MINIO_BUCKET", "zigma-data")  # Hive bucket
OBJECT_KEY = input("Enter destination object key (e.g., raw/people/new_upload.parquet): ").strip()

# Source: CSV file path or use an existing DataFrame variable name
src_choice = input("Upload from (1) CSV path or (2) existing DataFrame variable? Enter 1 or 2: ").strip()

if src_choice == "1":
    csv_path = input("Enter local CSV path to upload: ").strip()
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"{csv_path} not found.")
    df_src = pd.read_csv(csv_path)
elif src_choice == "2":
    var_name = input("Enter DataFrame variable name present in this notebook (e.g., 'preprocessed_data_a'): ").strip()
    if var_name not in globals():
        raise NameError(f"Variable '{var_name}' not found in globals.")
    df_src = globals()[var_name]
else:
    raise ValueError("Invalid choice. Enter 1 or 2.")

# Convert to Parquet in-memory (no preprocessing here)
table = pa.Table.from_pandas(df_src)
buf = BytesIO()
pq.write_table(table, buf, compression="snappy")
buf.seek(0)

# Connect to MinIO and ensure bucket exists
client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=MINIO_SECURE)
found = client.bucket_exists(BUCKET)
if not found:
    client.make_bucket(BUCKET)
    print(f"Created bucket: {BUCKET}")

# Upload Parquet
client.put_object(BUCKET, OBJECT_KEY, buf, length=len(buf.getvalue()), content_type="application/octet-stream")
print(f"Uploaded Parquet to s3://{BUCKET}/{OBJECT_KEY} on {MINIO_ENDPOINT}")
print("You can now query it via Trino (ensure your Hive/Metastore points to this bucket/prefix).")

In [ ]:
try:
    import trino
    from trino.dbapi import connect as trino_connect
    from trino.auth import BasicAuthentication
except Exception as e:
    raise ImportError("trino Python package is required. Add 'trino' to requirements and pip install it.")

# --- Connection config (edit defaults as needed) ---
TRINO_HOST = os.getenv("TRINO_HOST", "localhost")
TRINO_PORT = int(os.getenv("TRINO_PORT", "8080"))
TRINO_USER = os.getenv("TRINO_USER", "zigma_user")
TRINO_CATALOG = os.getenv("TRINO_CATALOG", "hive")  # Hive connector pointing to MinIO-backed data
TRINO_HTTP_SCHEME = os.getenv("TRINO_HTTP_SCHEME", "http")  # or "https"
# No auth required per your setup
auth = None

conn = trino_connect(
    host=TRINO_HOST,
    port=TRINO_PORT,
    user=TRINO_USER,
    catalog=TRINO_CATALOG,
    http_scheme=TRINO_HTTP_SCHEME,
    auth=auth,
)

cur = conn.cursor()

# --- List schemas and prompt user ---
cur.execute("SHOW SCHEMAS")
schemas = sorted([r[0] for r in cur.fetchall()])
print(f"Available schemas in catalog '{TRINO_CATALOG}':")
for i, s in enumerate(schemas):
    print(f"  {i+1}. {s}")

schema_input = input("Enter schema name (exact) from the list above: ").strip()
if schema_input not in schemas:
    raise ValueError(f"Schema '{schema_input}' not found in catalog {TRINO_CATALOG}")

# --- List tables in chosen schema ---
cur.execute(f"SHOW TABLES FROM {TRINO_CATALOG}.{schema_input}")
tables = sorted([r[0] for r in cur.fetchall()])
print(f"Tables in {TRINO_CATALOG}.{schema_input}:")
for i, t in enumerate(tables):
    print(f"  {i+1}. {t}")

raw_tables = input("Enter one or more table names (comma-separated) to dedupe: ").strip()
selected_tables = [t.strip() for t in raw_tables.split(",") if t.strip()]
if len(selected_tables) < 1:
    raise ValueError("Please provide at least 1 table name.")

# --- Load tables to pandas (optionally limit rows for speed) ---
row_limit = input("Optional row limit per table (blank for no limit): ").strip()
limit_sql = ""
if row_limit:
    try:
        limit_n = int(row_limit)
        limit_sql = f" LIMIT {limit_n}"
    except Exception:
        print("Invalid limit provided; loading full tables.")
        limit_sql = ""

loaded_dfs = []
for tbl in selected_tables:
    q = f"SELECT * FROM {TRINO_CATALOG}.{schema_input}.{tbl}{limit_sql}"
    print(f"Loading: {q}")
    cur.execute(q)
    rows = cur.fetchall()
    cols = [d[0] for d in cur.description]
    df_tbl = pd.DataFrame(rows, columns=cols)
    loaded_dfs.append((tbl, df_tbl))

# --- Map/clean using existing preprocess and combine logic ---
preprocessed = []
for tbl, df in loaded_dfs:
    print(f"Preprocessing table: {tbl} (rows={len(df):,})")
    preprocessed.append(preprocess_data(df))

# Concatenate any number of tables into one combined_df
combined_df = pd.concat(preprocessed, ignore_index=True)
combined_df["unique_id"] = combined_df.index.astype(str)

print(f"combined_df ready from Trino: {len(combined_df):,} rows; columns: {len(combined_df.columns)}")

In [ ]:
# Combine data
combined_df = combine_dataframes(preprocessed_data_a, preprocessed_data_b)

# Identify and show duplicates
duplicates = identify_and_show_duplicates(combined_df, subset_cols=["first_name", "last_name"])
print(f"Sample of duplicate records are: \n{duplicates} ")

Sample of duplicate records are: 
                                  full_name first_and_last_name first_name  \
1                       thomas of chudleigh    thomas chudleigh     thomas   
3                      thomas 1st chudleigh    thomas chudleigh     thomas   
4      thomas clifford, 1st baron chudleigh    thomas chudleigh     thomas   
5              thomas clifford of chudleigh    thomas chudleigh     thomas   
6                       tom baron chudleigh       tom chudleigh        tom   
...                                     ...                 ...        ...   
50569                        leonard horner      leonard horner    leonard   
50571                        leonard horner      leonard horner    leonard   
50573                        leonard horner      leonard horner    leonard   
50574                            len horner          len horner        len   
50577                            len horner          len horner        len   

       last_name         dob 

In [10]:
# Analyze completeness of data
db_api = DuckDBAPI()
analyze_completeness(combined_df, db_api)

alt.LayerChart(...)

In [11]:
# Profile the data
profile_data(combined_df, db_api)

alt.VConcatChart(...)

In [12]:
# --- Blocking Rule Utilities ---

def create_blocking_rules() -> List[Tuple[str, str]]:
    """Create and return a list of named blocking rules."""
    return [
        ("first_name & last_name", block_on("first_name", "last_name")),
        ("zip", block_on("zip")),
        ("state", block_on("state")),
        ("city", block_on("city")),
        ("first_name_metaphone & last_name_metaphone", block_on("first_name_metaphone", "last_name_metaphone")),
        ("last_name_metaphone", block_on("last_name_metaphone")),
        ("first char of first_name", block_on("substr(first_name, 1,1)")),
        ("phone", block_on("phone")),
    ]

def count_blocking_rule_comparisons(
    df: pd.DataFrame,
    blocking_rules: List[Tuple[str, str]],
    db_api: DuckDBAPI,
    link_type: str = "dedupe_only"
) -> dict:
    """Count comparisons for each blocking rule."""
    results = {}
    for name, rule in blocking_rules:
        count = count_comparisons_from_blocking_rule(
            table_or_tables=df,
            blocking_rule=rule,
            link_type=link_type,
            db_api=db_api,
        )
        results[name] = count
    return results

def print_comparison_counts(counts: dict):
    """Print the comparison counts for each rule."""
    for name, count in counts.items():
        print(f"{name}: {count}")

def analyze_worst_case_blocks(
    df: pd.DataFrame,
    rules: List[str],
    db_api: DuckDBAPI,
    top_n: int = 5
):
    """Analyze largest blocks for given rule combinations."""
    for rule in rules:
        result = n_largest_blocks(
            table_or_tables=df,
            blocking_rule=block_on(*rule.split(",")),
            link_type="dedupe_only",
            db_api=db_api,
            n_largest=top_n
        )
        print(f"\nTop {top_n} blocks for rule: {rule}")
        display(result.as_pandas_dataframe())  # Only works in Jupyter/Notebook environments


In [13]:
blocking_rules = create_blocking_rules()
comparison_counts = count_blocking_rule_comparisons(combined_df, blocking_rules, db_api)
print_comparison_counts(comparison_counts)

rules_to_analyze = [
    "first_name,last_name",
    "first_name,last_name,first_name_metaphone,last_name_metaphone"
]
analyze_worst_case_blocks(combined_df, rules_to_analyze, db_api)

Computation of number of comparisons post-filter conditions was skipped because the number of comparisons generated by your blocking rule exceeded max_rows_limit=1.00e+09.
It would be likely to be slow to compute.
If you still want to go ahead increase the value of max_rows_limit argument to above 2.558e+09.
Read more about the definitions here:
https://moj-analytical-services.github.io/splink/topic_guides/blocking/performance.html?h=filter+cond#filter-conditions


first_name & last_name: {'number_of_comparisons_generated_pre_filter_conditions': 579786, 'number_of_comparisons_to_be_scored_post_filter_conditions': 264604, 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."first_name" = r."first_name" AND l."last_name" = r."last_name"', 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}
zip: {'number_of_comparisons_generated_pre_filter_conditions': 130816972, 'number_of_comparisons_to_be_scored_post_filter_conditions': 65383197, 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."zip" = r."zip"', 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}
state: {'number_of_comparisons_generated_pre_filter_conditions': 2558134084, 'number_of_comparisons_to_be_scored_post_filter_conditions': 'exceeded max_rows_limit, see warning', 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."state" = r."state"', 'link_type_join_condition': 'where l."unique_id" < r."un

,key_0,key_1,count_l,count_r,block_count
0,sir,baronet,542,542,293764
1,william,\n,82,82,6724
2,john,\n,76,76,5776
3,\n,\n,67,67,4489
4,1st,bt.,57,57,3249



Top 5 blocks for rule: first_name,last_name,first_name_metaphone,last_name_metaphone


,key_0,key_1,key_2,key_3,count_l,count_r,block_count
0,sir,baronet,SR,PRNT,542,542,293764
1,william,\n,ALM,N,82,82,6724
2,john,\n,JN,N,76,76,5776
3,\n,\n,N,N,67,67,4489
4,1st,bt.,ST,PTT,57,57,3249


In [14]:
def create_settings():
    return SettingsCreator(
        link_type="dedupe_only",
        em_convergence=0.001,
        comparisons=[
            cl.JaroWinklerAtThresholds("first_name", [0.7, 0.85]),
            cl.JaroWinklerAtThresholds("last_name", [0.7, 0.85]),
            cl.ExactMatch("first_name_metaphone"),
            cl.ExactMatch("last_name_metaphone"),
            cl.EmailComparison("email"),
            cl.LevenshteinAtThresholds("phone", [0.7, 0.9]),
            cl.LevenshteinAtThresholds("street", [0.4, 0.7, 0.85]),
            cl.JaroWinklerAtThresholds("city", [0.4, 0.7, 0.85]),
            cl.ExactMatch("state"),
            cl.PostcodeComparison("zip"),
        ],
        blocking_rules_to_generate_predictions=[
            block_on("first_name", "last_name"),
            block_on("first_name_metaphone", "last_name_metaphone"),
            block_on("first_name", "zip"),
            block_on("last_name", "zip"),
            block_on("first_name_metaphone", "zip"),
            block_on("last_name_metaphone", "zip"),
            block_on("phone"),
        ],
        retain_intermediate_calculation_columns=False,
    )


In [15]:
def train_and_save_model(path: str,df: pd.DataFrame):
    import json
    settings = create_settings()

    # Train on a stratified small sample for speed, then apply to full data
    sample_size = min(len(df), 20000)  # cap training at 20k rows
    training_df = df.sample(n=sample_size, random_state=42) if len(df) > sample_size else df

    train_linker = Linker(training_df, settings, db_api=DuckDBAPI())
    # Use more selective training rules to avoid excessive comparisons
    rules = [
        block_on("first_name", "last_name"),
        "l.zip = r.zip and levenshtein(r.last_name, l.last_name) <= 2 and levenshtein(r.first_name, l.first_name) <= 2",
    ]
    train_linker.training.estimate_probability_two_random_records_match(rules, recall=0.6)
    train_linker.training.estimate_u_using_random_sampling(max_pairs=2000000)
    for br in [block_on("first_name", "last_name"), block_on("zip"), block_on("city")]:
        train_linker.training.estimate_parameters_using_expectation_maximisation(br)

    train_linker.misc.save_model_to_json(path, overwrite=True)

    # Build a full-data linker using the trained settings loaded from JSON
    with open(path, 'r', encoding='utf-8') as f:
        trained_settings = json.load(f)
    full_linker = Linker(df, trained_settings, db_api=DuckDBAPI())
    return full_linker


In [16]:
def visualize_model(linker):
    # Set renderer for VS Code/Notebook environments
    try:
        pio.renderers.default = "vscode"
    except Exception:
        pio.renderers.default = "notebook_connected"

    # Generate figures
    fig1 = linker.visualisations.match_weights_chart()
    fig2 = linker.visualisations.m_u_parameters_chart()
    fig3 = linker.visualisations.parameter_estimate_comparisons_chart()

    # Resize to avoid clipping
    for f in (fig1, fig2, fig3):
        try:
            f.update_layout(width=1400, height=800)
        except Exception:
            pass

    # Display
    display(fig1)
    display(fig2)
    display(fig3)



In [17]:
linker = train_and_save_model("./final_model1.json",combined_df)

Probability two random records match is estimated to be  0.000411.
This means that amongst all possible pairwise record comparisons, one in 2,431.49 are expected to match.  With 199,990,000 total possible comparisons, we expect a total of around 82,250.00 matching pairs
----- Estimating u probabilities using random sampling -----
u probability not trained for first_name - Jaro-Winkler distance of first_name >= 0.85 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
u probability not trained for last_name - Jaro-Winkler distance of last_name >= 0.85 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
u probability not trained for email - Exact match on email (comparison vector value: 4). This usually means the comparison level was never observed in the training data.
u probability not trained for email - Exact match on username (comparison vector value: 3). This usually

In [18]:
display_of_model_visualizations = visualize_model(linker)

alt.VConcatChart(...)

alt.HConcatChart(...)

alt.Chart(...)

In [19]:
def generate_predictions(linker, prediction_path: str, cluster_path: str,threshold: float):
    df_predictions = linker.inference.predict()
    df_predictions_pd = df_predictions.as_pandas_dataframe() 
    df_predictions_pd = df_predictions_pd[df_predictions_pd["match_probability"] > threshold]
    df_predictions_pd.to_csv(prediction_path, index=False)

    clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
        df_predictions, threshold_match_probability=threshold)
    clusters_pd = clusters.as_pandas_dataframe()
    clusters_pd.to_csv(cluster_path, index=False)
    return df_predictions_pd, clusters_pd


In [20]:
df_preds, clusters = generate_predictions(linker, "splink_predictions.csv", "splink_clusters.csv",0.99)

Blocking time: 1.47 seconds
Predict time: 10.44 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'first_name':
    u values not fully trained
Comparison: 'last_name':
    m values not fully trained
Comparison: 'last_name':
    u values not fully trained
Comparison: 'email':
    u values not fully trained
Comparison: 'phone':
    m values not fully trained
Comparison: 'phone':
    u values not fully trained
Comparison: 'street':
    m values not fully trained
Comparison: 'street':
    u values not fully trained
Comparison: 'city':
    m values not fully trained
Comparison: 'city':
    u values not fully trained
Comparison: 'state':
    m values not fully trained
Comparison: 'state':
    u values not fully trained


In [21]:
def get_deduped_id_mapping(df: pd.DataFrame) -> Dict[int, int]:
    """Create a mapping of deduplicated IDs, mapping unique_id_r to the smallest unique_id_l."""
    deduped_ids = df.groupby('unique_id_r')['unique_id_l'].first().reset_index()
    id_mapping = pd.Series(deduped_ids['unique_id_l'].values, index=deduped_ids['unique_id_r']).to_dict()
    int_id_mapping = {int(k): int(v) for k, v in id_mapping.items()}
    return int_id_mapping


def deduplicate_by_mapped_ids(df: pd.DataFrame,column_name:str,id_mapping,output_path:str) -> None:
    """Convert the 'unique_id' column to integer type if necessary."""
    if df[column_name].dtype != 'int64':
        df[column_name] = df[column_name].astype(int)
    df.loc[:, column_name] = df[column_name].replace(id_mapping)
    df = df.drop_duplicates(subset=column_name, keep='last')
    df.to_csv(output_path,index=False)


In [22]:
int_mapping = get_deduped_id_mapping(df_preds)
deduplicate_by_mapped_ids(combined_df,"unique_id",int_mapping,"merged.csv")

In [26]:
# Create a labeling template from predictions for manual labelling
# It saves unique_id_l, unique_id_r, and an empty 'match' column you can fill with 1 (match) or 0 (non-match)

def create_labels_template(df_predictions: pd.DataFrame, path: str = "labels_template.csv", n: int = 300):
    if df_predictions is None or df_predictions.empty:
        print("No predictions available. Run prediction first.")
        raise SystemExit
    cols_needed = {"unique_id_l", "unique_id_r", "match_probability"}
    missing = cols_needed - set(df_predictions.columns)
    if missing:
        print(f"Predictions missing required columns: {missing}")
        raise SystemExit
    # sample top-N high-confidence pairs to make labelling easier
    sample = (
        df_predictions[["unique_id_l", "unique_id_r", "match_probability"]]
        .drop_duplicates(subset=["unique_id_l", "unique_id_r"])  # ensure unique pairs
        .sort_values("match_probability", ascending=False)
        .head(n)
        .copy()
    )
    sample["match"] = ""  # leave empty for you to fill 1/0
    sample[["unique_id_l", "unique_id_r", "match"]].to_csv(path, index=False)
    print(f"Saved labelling template to: {path}. Fill 'match' with 1/0 and save as a new file.")

# Generate the template (adjust n if you want more/less rows)
create_labels_template(df_preds, path="labels_template.csv", n=300)


Saved labelling template to: labels_template.csv. Fill 'match' with 1/0 and save as a new file.


In [ ]:
# If predictions are not in memory, load from disk and create labels template
preds_path = "splink_predictions.csv"
if 'df_preds' not in globals() or df_preds is None or df_preds.empty:
    if os.path.exists(preds_path):
        df_preds = pd.read_csv(preds_path)
        print(f"Loaded predictions from: {preds_path} ({len(df_preds):,} rows)")
    else:
        raise FileNotFoundError(f"{preds_path} not found. Please run predictions first.")

# Create a labels template from predictions (you fill 'match' = 1/0)
create_labels_template(df_preds, path="labels_template.csv", n=300)


In [39]:
# Load a user-labelled CSV and run accuracy analysis (custom, no Splink table APIs)
# Expected columns in labels: unique_id_l, unique_id_r, match (1/0)

labels_csv_path = "labels_filled.csv"  # change if needed
if os.path.exists(labels_csv_path):
    labels_df = pd.read_csv(labels_csv_path)
    missing = {"unique_id_l", "unique_id_r", "match"} - set(labels_df.columns)
    if missing:
        raise ValueError(f"Labels file missing columns: {missing}")

    # Ensure dtypes
    labels_df["unique_id_l"] = labels_df["unique_id_l"].astype(str)
    labels_df["unique_id_r"] = labels_df["unique_id_r"].astype(str)
    labels_df["match"] = pd.to_numeric(labels_df["match"], errors="coerce").fillna(0).astype(int)

    # Ensure predictions in memory
    if 'df_preds' not in globals() or df_preds is None or df_preds.empty:
        if os.path.exists("splink_predictions.csv"):
            df_preds = pd.read_csv("splink_predictions.csv")
        else:
            raise FileNotFoundError("splink_predictions.csv not found. Run predictions first.")

    # Merge labels with predictions on pair ids
    preds_pairs = df_preds[["unique_id_l", "unique_id_r", "match_probability"]].copy()
    preds_pairs["unique_id_l"] = preds_pairs["unique_id_l"].astype(str)
    preds_pairs["unique_id_r"] = preds_pairs["unique_id_r"].astype(str)

    data = labels_df.merge(preds_pairs, on=["unique_id_l", "unique_id_r"], how="left")
    data["match_probability"] = data["match_probability"].fillna(0.0)

    # Compute precision/recall/F1 across thresholds
    thresholds = np.linspace(0.0, 0.999, 50)
    rows = []
    for t in thresholds:
        pred = (data["match_probability"] >= t).astype(int)
        tp = int(((pred == 1) & (data["match"] == 1)).sum())
        fp = int(((pred == 1) & (data["match"] == 0)).sum())
        fn = int(((pred == 0) & (data["match"] == 1)).sum())
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
        rows.append({"threshold": t, "tp": tp, "fp": fp, "fn": fn, "precision": precision, "recall": recall, "f1": f1})

    acc_df = pd.DataFrame(rows)
    best = acc_df.sort_values("f1", ascending=False).head(1)
    print("Best threshold by F1:")
    display(best)

    # Melt for chart
    chart_df = acc_df.melt(id_vars=["threshold"], value_vars=["precision", "recall", "f1"], var_name="metric", value_name="value")
    fig = px.line(chart_df, x="threshold", y="value", color="metric", title="Precision/Recall/F1 vs Threshold")
    try:
        fig.update_layout(width=1200, height=600)
    except Exception:
        pass
    display(fig)
else:
    print("No labels file found. Fill 'labels_template.csv' and save as 'labels_filled.csv' to run accuracy analysis.")


Best threshold by F1:


,threshold,tp,fp,fn,precision,recall,f1
0,0.0,0,300,0,0.0,0.0,0.0


In [1]:
# --- Robust Interactive Record Lookup using Existing Predictions (safe, no UnboundLocalError) ---

def find_similar_records_for_new_input(new_record: dict = None, interactive: bool = False):
    """
    Use existing predictions/clusters to:
    - Find most similar pairs by match_probability
    - Show clusters containing those records
    - Assess likelihood of duplicate vs unique

    Params:
    - new_record: optional dict with keys first_name,last_name,email,phone,address,city,zip
    - interactive: if True, prompt with input(); otherwise use new_record or defaults
    """
    
    # Access globals safely
    df_preds_local = globals().get('df_preds', None)
    clusters_local = globals().get('clusters', None)
    combined_df_local = globals().get('combined_df', None)

    # Load predictions if not present
    if df_preds_local is None or getattr(df_preds_local, 'empty', True):
        if os.path.exists('splink_predictions.csv'):
            df_preds_local = pd.read_csv('splink_predictions.csv')
            print(f"Loaded predictions: {len(df_preds_local):,} rows")
        else:
            print("splink_predictions.csv not found. Please run predictions first.")
            return

    # Load clusters if not present
    if clusters_local is None or getattr(clusters_local, 'empty', True):
        if os.path.exists('splink_clusters.csv'):
            clusters_local = pd.read_csv('splink_clusters.csv')
            print(f"Loaded clusters: {len(clusters_local):,} rows")
        else:
            print("splink_clusters.csv not found. Please run clustering first.")
            return

    # Ensure combined_df is available for record detail display
    if combined_df_local is None or getattr(combined_df_local, 'empty', True):
        if os.path.exists('merged.csv'):
            try:
                combined_df_local = pd.read_csv('merged.csv')
                print("Loaded 'merged.csv' for record detail display.")
            except Exception:
                print("combined_df not found in memory. Please run earlier cells to build it.")
                return
        else:
            print("combined_df not found in memory. Please run earlier cells to build it.")
            return

    # Validate required columns in predictions
    required_pred_cols = {"unique_id_l", "unique_id_r", "match_probability"}
    missing_pred = required_pred_cols - set(df_preds_local.columns)
    if missing_pred:
        print(f"Predictions missing columns: {missing_pred}")
        return

    # Coerce ID columns to numeric safely
    for c in ["unique_id_l", "unique_id_r"]:
        df_preds_local[c] = pd.to_numeric(df_preds_local[c], errors='coerce')
    df_preds_local = df_preds_local.dropna(subset=["unique_id_l", "unique_id_r"]).copy()
    df_preds_local["unique_id_l"] = df_preds_local["unique_id_l"].astype(int)
    df_preds_local["unique_id_r"] = df_preds_local["unique_id_r"].astype(int)

    # Coerce combined_df unique_id for joining/lookup
    if "unique_id" not in combined_df_local.columns:
        print("combined_df is missing 'unique_id' column. Please re-run combine step.")
        return
    combined_df_local["_unique_id_int"] = pd.to_numeric(combined_df_local["unique_id"], errors='coerce').astype('Int64')

    # Build new_record
    if new_record is None:
        new_record = {}
    def get_val(key, prompt, default=""):
        if interactive:
            return input(prompt).strip()
        return str(new_record.get(key, default)).strip()

    print("\n=== New Record (interactive=" + ("True" if interactive else "False") + ") ===")
    nr = {}
    nr['first_name'] = get_val('first_name', "First Name: ").lower()
    nr['last_name']  = get_val('last_name',  "Last Name: ").lower()
    nr['email']      = get_val('email',      "Email: ").lower()
    nr['phone']      = get_val('phone',      "Phone: ")
    nr['address']    = get_val('address',    "Address: ").lower()
    nr['city']       = get_val('city',       "City: ").lower()
    nr['zip']        = get_val('zip',        "ZIP/Postcode: ").upper()

    # Derived fields
    nr['full_name'] = (nr['first_name'] + ' ' + nr['last_name']).strip()
    nr['first_name_metaphone'] = doublemetaphone(nr['first_name'])[0] if nr['first_name'] else ''
    nr['last_name_metaphone']  = doublemetaphone(nr['last_name'])[0] if nr['last_name'] else ''
    nr['email_domain'] = nr['email'].split('@')[1] if '@' in nr['email'] else ''

    print(f"Searching for similar to: {nr.get('full_name','')} ({nr.get('email','')})")

    # Blocking candidates
    candidate_ids = set()
    if nr['first_name'] and nr['last_name']:
        m = combined_df_local[(combined_df_local.get('first_name','') == nr['first_name']) &
                              (combined_df_local.get('last_name','')  == nr['last_name'])]
        candidate_ids.update(m["_unique_id_int"].dropna().astype(int).tolist())
    if nr['first_name_metaphone'] and nr['last_name_metaphone']:
        m = combined_df_local[(combined_df_local.get('first_name_metaphone','') == nr['first_name_metaphone']) &
                              (combined_df_local.get('last_name_metaphone','')  == nr['last_name_metaphone'])]
        candidate_ids.update(m["_unique_id_int"].dropna().astype(int).tolist())
    if nr['zip']:
        m = combined_df_local[combined_df_local.get('zip','') == nr['zip']]
        candidate_ids.update(m["_unique_id_int"].dropna().astype(int).tolist())
    if nr['email_domain']:
        m = combined_df_local[combined_df_local.get('email_domain','') == nr['email_domain']]
        candidate_ids.update(m["_unique_id_int"].dropna().astype(int).tolist())

    if not candidate_ids:
        print("No similar records found via blocking.")
        return

    preds_mask = df_preds_local['unique_id_l'].isin(candidate_ids) | df_preds_local['unique_id_r'].isin(candidate_ids)
    candidate_predictions = df_preds_local.loc[preds_mask].copy()
    if candidate_predictions.empty:
        print("No prediction pairs found for the candidates.")
        return

    candidate_predictions = candidate_predictions.sort_values('match_probability', ascending=False)

    print("\nTop matches:")
    for _, row in candidate_predictions.head(10).iterrows():
        prob = row['match_probability']
        id_l = int(row['unique_id_l'])
        id_r = int(row['unique_id_r'])
        rec_l = combined_df_local.loc[combined_df_local['_unique_id_int'] == id_l].head(1)
        rec_r = combined_df_local.loc[combined_df_local['_unique_id_int'] == id_r].head(1)
        print(f"  p={prob:.4f}  L={id_l}  R={id_r}")
        if not rec_l.empty:
            rl = rec_l.iloc[0]
            print(f"    L: {rl.get('full_name','')} | {rl.get('email','')} | {rl.get('phone','')}")
        if not rec_r.empty:
            rr = rec_r.iloc[0]
            print(f"    R: {rr.get('full_name','')} | {rr.get('email','')} | {rr.get('phone','')}")

    print("\nClusters containing candidates:")
    if 'cluster_id' not in clusters_local.columns or 'unique_id' not in clusters_local.columns:
        print("Clusters file missing 'cluster_id' or 'unique_id' columns.")
        return
    clusters_local['_unique_id_int'] = pd.to_numeric(clusters_local['unique_id'], errors='coerce').astype('Int64')
    cluster_ids = clusters_local.loc[clusters_local['_unique_id_int'].isin(list(candidate_ids)), 'cluster_id'].dropna().unique().tolist()

    if not cluster_ids:
        print("  None")
    else:
        print(f"  Found {len(cluster_ids)}")
        for cid in cluster_ids[:5]:
            members = clusters_local.loc[clusters_local['cluster_id'] == cid, '_unique_id_int'].dropna().astype(int).tolist()
            details = combined_df_local.loc[combined_df_local['_unique_id_int'].isin(members)]
            print(f"  - Cluster {cid} ({len(members)} members)")
            for _, rec in details.head(5).iterrows():
                print(f"      {rec.get('unique_id','')}: {rec.get('full_name','')} | {rec.get('email','')} | {rec.get('phone','')}")
            if len(details) > 5:
                print(f"      ... and {len(details) - 5} more")

    if not candidate_predictions.empty:
        max_prob = float(candidate_predictions['match_probability'].max())
        threshold = 0.8
        print("\nDuplicate assessment:")
        print(f"  Highest match probability: {max_prob:.4f}")
        if max_prob >= threshold:
            print("  LIKELY DUPLICATE")
        else:
            print("  LIKELY UNIQUE")

    return candidate_predictions, cluster_ids

# Example non-interactive call (edit values as needed) so Run All doesn't block
# Comment this out if you want interactive prompts instead.
find_similar_records_for_new_input({
    'first_name': 'thomas',
    'last_name': 'chudleigh',
    'email': 'thomas@example.com',
    'phone': '',
    'address': '',
    'city': 'devon',
    'zip': 'TQ138DF'
}, interactive=False)


NameError: name 'os' is not defined